In [2]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 50.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 24.6 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install  plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 66.0 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import tensorflow.keras.optimizers.schedules as schedules

D0610 03:52:21.196883462      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0610 03:52:21.196928195      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0610 03:52:21.196931984      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0610 03:52:21.196934745      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0610 03:52:21.196937162      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0610 03:52:21.196939865      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0610 03:52:21.196942410      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0610 03:52:21.

In [5]:
#데이터 불러오기
epochs=100
rootPath="../input/plant-pathology-2020-fgvc7/"
images= "images/"
test= "test.csv"
train = "train.csv"
result = "sample_submission.csv"

submission = pd.read_csv(rootPath+result)
testData = pd.read_csv(rootPath+test)
trainData = pd.read_csv(rootPath+train)
plusData=pd.read_csv("/kaggle/input/pluslabel/plus.csv")


In [6]:
#데이터불러오는 함수
def originPath(name):
    return rootPath + 'images/' + name + '.jpg'
#추가 데이터불러오는 함수
def plusPath(name):
    return "../input/plusdata/" + name + '.jpg'

In [7]:
#데이터불러오는코드
testPaths = testData.image_id.apply(originPath).values
trainPaths = trainData.image_id.apply(originPath).values
plusPaths = plusData.image_id.apply(plusPath).values
trainLabels = np.float32(trainData.loc[:, 'healthy':'scab'].values)
plusLabels = np.float32(plusData.loc[:, 'healthy':'scab'].values)
trainPaths, validPaths, trainLabels, validLabels =train_test_split(trainPaths, trainLabels, test_size=0.15)

In [8]:
#원래데이터에 추가 데이터 합치기
trainLabels=np.concatenate((trainLabels,plusLabels))

In [9]:
#원래데이터 경로에 추가경로 합치기
trainPaths=np.concatenate((trainPaths,plusPaths))

In [10]:
#2048 1365 3 데이터의 shape

In [11]:
1365//4#데이터가 큼으로 비율대로 자르기

341

In [12]:
trainLabels.shape

(2546, 4)

In [13]:
height=341
width=514

In [14]:
#데이터 전처리 함수
def dataPreprocessing(filename, label=None, image_size=(width, height)):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    if label is None:
        return image
    return image, label
    
#데이터 증가 함수
def dataAugment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if label is None:    
        return image
    return image, label

In [15]:
#TPU설정
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
#TPU상태에따른 바치사이즈 조정
batchSize = 16 * strategy.num_replicas_in_sync

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


In [16]:
#이미지 불러옴
trainDataset = (
    tf.data.Dataset
    .from_tensor_slices((trainPaths, trainLabels))
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .map(dataAugment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(batchSize)
    .prefetch(AUTO)
)

validDataset = (
    tf.data.Dataset
    .from_tensor_slices((validPaths, validLabels))
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .batch(batchSize)
    .cache()
    .prefetch(AUTO)
)

testDataset = (
    tf.data.Dataset
    .from_tensor_slices(testPaths)
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .batch(batchSize)
)


In [17]:
from tensorflow.keras.models import clone_model
class Combination():
    
    def __init__(self):
        self.model=None
        self.optimizer=None
    
    def setModel(self,Model):
        model=Model
        self.model=model.getModel()
        print(self.model.summary())
    
    def setOptimizer(self,Optimizer):
        
        self.optimizer=Optimizer
        print(self.optimizer)
    
    
        
        
    
    def onLearning(self,epochs=30):
        
        with strategy.scope():
        


            
            self.model.compile(optimizer=self.optimizer,
                          loss = 'categorical_crossentropy',
                          metrics=['categorical_accuracy'])
        
        stepPerEpoch = trainLabels.shape[0] // batchSize
        history = self.model.fit(
                    trainDataset,
                    epochs=epochs,
                    steps_per_epoch=stepPerEpoch,
                    validation_data=validDataset)
        return history

In [18]:
class Model:
    def __init__(self):
        pass
    def getModel():
        pass

In [19]:
class Optimizer:
    def __init__(self):
        pass
    def getOptimizer():
        pass

In [20]:
#모델 불러오기
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.applications import ResNet152V2

In [21]:
class Vgg16(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([
                            L.BatchNormalization(input_shape=(width, height, 3)),
                            VGG16(input_shape=(width, height, 3),
                                                      weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax',kernel_initializer='he_normal')])
        
    def getModel(self):
        return self.Model
    

In [22]:
class Vgg19(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([VGG19(input_shape=(width, height, 3),
                                                    weights="imagenet",
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [23]:
class Inceptionv3(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([InceptionV3(
                                                    weights="imagenet",
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.3),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax',kernel_initializer='he_normal')])
        
    def getModel(self):
        return self.Model
    

In [24]:
class Resnet50(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet50(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [25]:
class Resnet101(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet101(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [26]:
class Resnet152(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet152(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [27]:
class Resnet50v2(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet50V2(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [28]:
class Resnet101v2(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet101V2(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.5),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [29]:
class Resnet152v2(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet152V2(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.5),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [30]:
class SGD(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.SGD(momentum=0.9,learning_rate=schedules)
        return self.Optim

In [31]:
class Adagrad(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.Adagrad(epsilon=1e-6,learning_rate=schedules)
        return self.Optim

In [32]:
class RMSprop(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.RMSprop(rho=0.9, epsilon=1e-06,learning_rate=schedules)
        return self.Optim
    
        

In [33]:
class Adam(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999,learning_rate=schedules)
        return self.Optim

In [34]:
# models={"Vgg16":Vgg16(),"Vgg19":Vgg19(),"Inceptionv3":Inceptionv3()\
#         ,"Resnet50":Resnet50(),"Resnet101":Resnet101(),"Resnet152":Resnet152()\
#         ,"Resnet50v2":Resnet50v2(),"Resnet101v2":Resnet101v2(),"Resnet152v2":Resnet152v2()}

In [35]:
# optimizers={"SGD":SGD(),"Adagrad":Adagrad(),"RMSprop":RMSprop(),"Adam":Adam()}

In [36]:
# learningRateSchedulers={"CosineDecay":schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=1000, alpha=0.0)\
#                        ,"CosineDecayRestarts":schedules.CosineDecayRestarts(initial_learning_rate=0.001, t_mul=2.0,m_mul=1.0,first_decay_steps=1000, alpha=0.001)\
#                        ,"ExponentialDecay":schedules.ExponentialDecay(initial_learning_rate=0.01,decay_steps=50,decay_rate=0.96,staircase=True)\
#                        ,"InverseTimeDecay":schedules.InverseTimeDecay(initial_learning_rate = 0.01,decay_steps = 1.0,decay_rate = 0.5)
#                        }

In [37]:
models={"Inceptionv3":Inceptionv3(),"Resnet101v2":Resnet101v2()}

171317808/171317808 [==============================] - 1s 0us/step


In [38]:
optimizers={"SGD":SGD()}

In [39]:
learningRateSchedulers={"CosineDecayRestarts":schedules.CosineDecayRestarts(initial_learning_rate=0.001, t_mul=2.0,m_mul=1.0,first_decay_steps=1000, alpha=0.001)}

In [40]:
combination=Combination()

In [41]:
def display(training, validation, title,yTitle,epochs):     
        fig = go.Figure()

        fig.add_trace(go.Scatter(x=np.arange(1, epochs+1), mode='lines+markers', y=training, marker=dict(color="#dc143c"),name="Train"))

        fig.add_trace(
            go.Scatter(x=np.arange(1, epochs+1), mode='lines+markers', y=validation, marker=dict(color="#0080ff"),
                   name="Validation"))

        fig.update_layout(title_text=title, yaxis_title=yTitle, xaxis_title="Epochs", template="plotly_dark")
        fig.show()

In [42]:

def performanceCheck(epochs):
    for modelName,model in models.items():
#         weights=model.getModel().get_weights()
        for optimizerName,optimizer in optimizers.items():
            for learningRateSchedulerName,learningRateScheduler in learningRateSchedulers.items():
#                 model.getModel().set_weights(weights)
                combination.setModel(model)
                combination.setOptimizer(optimizer.setSchedules(learningRateScheduler))
                history=combination.onLearning(epochs=epochs)
                trainAcc=history.history['categorical_accuracy']
                evalAcc=history.history['val_categorical_accuracy']
                trainLoss=history.history['loss']
                evalLoss=history.history['val_loss']
                title=modelName+" "+optimizerName+" "+learningRateSchedulerName+" "
                display(trainAcc,evalAcc,title+"Accuracy","Accuracy",epochs)
                display(trainLoss,evalLoss,title+"Loss","Loss",epochs)

In [43]:
performanceCheck(30)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, None, None, 2048)  21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 4)                 8196      
                                                                 
Total params: 21,810,980
Trainable params: 21,776,548
Non-trainable params: 34,432
_________________________________________________________________
None
Epoch 1/30


2023-06-10 03:54:09.216458: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-06-10 03:54:09.830044: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


19/19 [==============================] - ETA: 0s - loss: 1.3575 - categorical_accuracy: 0.3516

2023-06-10 03:55:23.514984: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-06-10 03:55:23.832619: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


19/19 [==============================] - 135s 3s/step - loss: 1.3575 - categorical_accuracy: 0.3516 - val_loss: 1.1679 - val_categorical_accuracy: 0.4745
Epoch 2/30
19/19 [==============================] - 10s 531ms/step - loss: 1.2332 - categorical_accuracy: 0.4823 - val_loss: 0.9829 - val_categorical_accuracy: 0.7336
Epoch 3/30
19/19 [==============================] - 9s 491ms/step - loss: 1.0345 - categorical_accuracy: 0.6188 - val_loss: 0.6806 - val_categorical_accuracy: 0.8431
Epoch 4/30
19/19 [==============================] - 11s 593ms/step - loss: 0.8397 - categorical_accuracy: 0.7052 - val_loss: 0.5487 - val_categorical_accuracy: 0.8431
Epoch 5/30
19/19 [==============================] - 10s 526ms/step - loss: 0.6914 - categorical_accuracy: 0.7496 - val_loss: 0.4939 - val_categorical_accuracy: 0.8686
Epoch 6/30
19/19 [==============================] - 9s 484ms/step - loss: 0.6822 - categorical_accuracy: 0.7492 - val_loss: 0.4544 - val_categorical_accuracy: 0.8796
Epoch 7/30
19

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101v2 (Functional)    (None, 17, 11, 2048)      42626560  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 4)                 8196      
                                                                 
Total params: 42,634,756
Trainable params: 42,537,092
Non-trainable params: 97,664
_________________________________________________________________
None
Epoch 1/30


2023-06-10 04:01:15.757060: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-06-10 04:01:16.640682: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


19/19 [==============================] - ETA: 0s - loss: 1.5281 - categorical_accuracy: 0.3104

2023-06-10 04:02:16.986371: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-06-10 04:02:17.383824: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


19/19 [==============================] - 112s 2s/step - loss: 1.5281 - categorical_accuracy: 0.3104 - val_loss: 1.2059 - val_categorical_accuracy: 0.4891
Epoch 2/30
19/19 [==============================] - 10s 548ms/step - loss: 1.2129 - categorical_accuracy: 0.4856 - val_loss: 0.9957 - val_categorical_accuracy: 0.7226
Epoch 3/30
19/19 [==============================] - 10s 518ms/step - loss: 0.9517 - categorical_accuracy: 0.6369 - val_loss: 0.6501 - val_categorical_accuracy: 0.7920
Epoch 4/30
19/19 [==============================] - 10s 509ms/step - loss: 0.7471 - categorical_accuracy: 0.7208 - val_loss: 0.5837 - val_categorical_accuracy: 0.8212
Epoch 5/30
19/19 [==============================] - 11s 575ms/step - loss: 0.6520 - categorical_accuracy: 0.7574 - val_loss: 0.5254 - val_categorical_accuracy: 0.8394
Epoch 6/30
19/19 [==============================] - 10s 518ms/step - loss: 0.5773 - categorical_accuracy: 0.7891 - val_loss: 0.4531 - val_categorical_accuracy: 0.8759
Epoch 7/30


In [46]:
model=models['Resnet101v2'].getModel()

In [47]:
testModel=clone_model(model)

In [48]:
y=testModel.predict(testDataset)

15/15 [==============================] - 139s 9s/step


In [49]:
submission=pd.read_csv("/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv")
submission

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.25,0.25,0.25,0.25
1,Test_1,0.25,0.25,0.25,0.25
2,Test_2,0.25,0.25,0.25,0.25
3,Test_3,0.25,0.25,0.25,0.25
4,Test_4,0.25,0.25,0.25,0.25
...,...,...,...,...,...
1816,Test_1816,0.25,0.25,0.25,0.25
1817,Test_1817,0.25,0.25,0.25,0.25
1818,Test_1818,0.25,0.25,0.25,0.25
1819,Test_1819,0.25,0.25,0.25,0.25


In [50]:
result=pd.DataFrame({
    "image_id":submission['image_id'],
    "healthy":y[:,0],
    "multiple_diseases":y[:,1],
    "rust":y[:,2],
    "scab":y[:,3]
})

In [52]:
result.to_csv("submission.csv",index=False)